In [22]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from lib import *

from lib.image_processing import load_tiff_stack, moving_average_subtraction, identify_particles
from lib.particle_tracking import extract_roi, track_particles
from lib.visualization import ParticleVisualizer
from lib.config import directory_setup

In [ ]:
dir_Path = './'
Path, ID, X, Y, Sframe, Eframe = f'{dir_Path}/Streams/Cell1_S1.tif', 'P1_C1_S1_11', 685, 660, 1, 300
stackPath, stackFile = os.path.split(Path)
startColX = X
startRowY = Y
particleID = ID
SFrame = Sframe
EFrame = Eframe

In [21]:
directory_setup(stackPath, stackFile, particleID, startColX, startRowY, X, Y)

In [ ]:
# Load and process the stack
print("Loading image stack...")
stack = load_tiff_stack(Path, Sframe, Eframe)
print(f"Stack loaded with shape: {stack.shape}")

In [ ]:
# Perform background subtraction
print("Performing background subtraction...")
subtracted_matrix, subtracted_matrix_b, subtracted_matrix_d, subtracted_matrix_double = \
    moving_average_subtraction(stack, FRAME_RANGE)

In [ ]:
# Identify particles
print("Identifying particles...")
particle_region, particle_region_b, particle_region_d = \
    identify_particles(subtracted_matrix, subtracted_matrix_b, subtracted_matrix_d, 
                      THRESH_STACK, PARTICLE_SIZE_THRESH)

In [ ]:
# Extract ROI and track particles
print("Tracking particles...")
particle_region_roi = extract_roi(particle_region, Y, X, ROI)
particle_centroid_roi, updated_particle_region_roi, jump_frames = \
    track_particles(particle_region_roi, PARTICLE_DEFAULT_DIAMETER, JUMP_THRESH, MIN_AREA)

print(f"Tracking complete. {jump_frames} frames had jumps that were corrected.")

In [ ]:
# Plot some results
print("Plotting sample frames...")

p = ParticleVisualizer(save_path=, figsize=(10,8), dpi=300)

for i in range(0, min(10, stack.shape[0])):
    # Plot original frame ROI
    p.plot_frame(stack[i, Y-ROI:Y+ROI+1, X-ROI:X+ROI+1], f"Original Frame {i+1}")
    
    # Plot subtracted frame ROI
    p.plot_frame(subtracted_matrix[i, Y-ROI:Y+ROI+1, X-ROI:X+ROI+1], f"Subtracted Frame {i+1}")
    
    # Plot particle mask
    p.plot_frame(updated_particle_region_roi[i], f"Particle Mask Frame {i+1}")


In [ ]:
# Save tracking results to disk
print("Saving results...")
# np.save(f"{save_base_name}_centroids.npy", particle_centroid_roi)
# np.save(f"{save_base_name}_masks.npy", updated_particle_region_roi)

print("Processing complete!")